#  使用数据拆分器

当我们使用机器学习，我们需要将数据分为训练集、验证机和测试集，MoleculeNet 加载器能够自动帮我们完成这些。但是你应该如何拆分数据？这个问题起初似乎很简单，但事实证明它相当复杂。有许多拆分数据的方法，你选择哪种方法会对你的结果的可靠性产生很大的影响。本教程介绍了 DeepChem 提供的一些拆分方法。

## Colab

本教程和目录中的其余部分都是在 Google colab 中完成。如果您想在 colab 中打开此笔记本，您可以点击以下链接。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BioGavin/deepchem/blob/master/examples/tutorials-zh/Working_With_Splitters.ipynb)



In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

## 拆分器

在 DeepChem 中，将样品分割成多个数据集的方法是由 `Splitter` 对象定义的。为你的数据选择一个合适的方法是非常重要的，否则，你的训练模型可能看起来比它的实际效果好得多。

考虑一个典型的药物开发流程。首先，你可能会筛选成千上万的分子，看它们是否与你感兴趣的目标结合。一旦你找到一个似乎有效的分子，你就试图通过数以千计的微小变化来优化它，寻找一个结合更强的分子。然后，你可能在动物身上测试它，发现它有不可接受的毒性，所以你尝试更多的变化来解决这些问题。

这对化学数据集来说会有一个重要的后果：它们往往包括很多相互之间非常相似的分子。如果你以一种天真的方式将数据分成训练集和测试集，训练集将包括许多与测试集中的分子非常相似的分子，即使它们并不完全相同。因此，该模型在测试集上可能做得非常好，但当你试图将其用于与训练数据不太相似的其他数据时，就会严重失败。

让我们看看 DeepChem 中提供的一些数据拆分方法。

### RandomSplitter

这是最简单的拆分器之一。它只是以完全随机的方式为训练、验证和测试集选择样本。

我们刚才不是说这是个坏主意吗？嗯，这取决于你的数据。如果每个样本都是真正独立的，那么这也是拆分数据的一个好方法。没有普遍意义上的最佳拆分器的选择，这完全取决于你的特定数据集，而对于某些数据集来说，这是一个很好的选择。

### RandomStratifiedSplitter

有些数据集是非常不平衡的，所有样本中只有很小一部分是阳性的。在这种情况下，随机拆分有时可能会导致验证集或测试集在某些任务中只有很少甚至没有阳性样本，这使得它无法评估性能。

`RandomStratifiedSplitter` 通过平均分配正负样本来解决这个问题。如果你要求80/10/10的分割，验证集和测试集不仅包含10%的样本，而且还包含每个任务的10%的阳性样本。

### ScaffoldSplitter

这个拆分器试图解决上面讨论的问题，即许多分子之间非常相似。它确定了构成每个分子核心的骨架，并确保将所有具有相同骨架的分子放入同一个数据集。这仍然不是一个完美的解决方案，因为两个分子可能有不同的脚手架，但在其他方面却非常相似，但它通常比随机拆分有很大的改进。

### ButinaSplitter

这是另一个试图解决类似分子问题的拆分器。它根据分子指纹对它们进行聚类，因此具有相似指纹的分子将倾向于在同一个数据集中。这种拆分算法所需的时间与分子数量的平方成正比，所以它主要适用于中小尺寸的数据集。

### SpecifiedSplitter

这个拆分器把一切都留给了用户，你准确地告诉它在每个数据集中要放哪些样本。当你事先知道一个特定的分割方式适合于你的数据时，这很有用。

一个例子是时间上的分割。假设一个研究项目，你不断地产生和测试新的分子。随着你获得数据越来越多，你定期在稳步增长的数据集上重新训练你的模型，然后用它来预测其他尚未测试的分子的结果。验证这种方法是否有效的一个好方法是选择一个特定的截止日期，在当前的所有数据上训练模型，然后看看它对后来产生的其他数据的预测效果如何。

## Effect of Using Different Splitters

我们来看一个例子。我们将使用 RandomSplitter、ScaffoldSplitter 和 ButinaSplitter 来拆分 Tox21 毒性数据集。 对于每一个，我们都训练一个模型，并在训练和测试集上进行评估。

In [1]:
import deepchem as dc

splitters = ['random', 'scaffold', 'butina']
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
for splitter in splitters:
    tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP', split=splitter)
    train_dataset, valid_dataset, test_dataset = datasets
    model = dc.models.MultitaskClassifier(n_tasks=len(tasks), n_features=1024, layer_sizes=[1000])
    model.fit(train_dataset, nb_epoch=10)
    print('splitter:', splitter)
    print('training set score:', model.evaluate(train_dataset, [metric], transformers))
    print('test set score:', model.evaluate(test_dataset, [metric], transformers))
    print()

splitter: random
training set score: {'roc_auc_score': 0.9560766203173238}
test set score: {'roc_auc_score': 0.8088861019955839}

splitter: scaffold
training set score: {'roc_auc_score': 0.9582835670901536}
test set score: {'roc_auc_score': 0.6803307954037949}

splitter: butina
training set score: {'roc_auc_score': 0.9578120869103354}
test set score: {'roc_auc_score': 0.6057007877463954}



所有这些都在训练集上产生了非常相似的性能，但 RandomSplitter 在测试集上的性能要高得多。ScaffoldSplitter 的测试集得分较低，而 ButinaSplitter 则更低。这是否意味着 RandomSplitter 的效果更好？不！这意味着 RandomSplitter 并不能给你一个准确的衡量你模型工作情况的标准。因为测试集包含很多与训练集中的分子非常相似的分子，它不是真正独立的。它使模型看起来比它实际工作要好。 ScaffoldSplitter 和 ButinaSplitter 可以更好地指示在未知的独立数据上可以期待什么。

# 恭喜！是时候加入社区了！

恭喜您完成本教程笔记本！如果您喜欢本教程并希望继续使用 DeepChem，我们鼓励您完成本系列的其余教程。您还可以通过以下方式帮助 DeepChem 社区：

## 在 [GitHub](https://github.com/deepchem/deepchem) 上为 DeepChem 点亮小星星
这有助于大家建立对 DeepChem 项目和我们正在尝试构建的开源药物发现工具的共识。

## 加入 DeepChem Gitter
DeepChem [Gitter](https://gitter.im/deepchem/Lobby) 聚集了许多对生命科学深度学习感兴趣的科学家、开发人员和爱好者，欢迎加入！